# Challenge: Classification

In this challenge, you will use a quantum circuit as a classifier, to classify input data into two classes.

The data will be 'quantum', meaning that your circuit will receive quantum states as an input.

Depending on some property of these states, the classifier will decide whether they should receive the label '0', or '1'.

This can be achieved with variational circuits, and we can use the following as an [example](https://arxiv.org/abs/1802.06002)

The idea is to use an 'ancilla' qubit, which, when measured will decide whether a given input gets classified as a zero or a one. 

The measurement of a quantum state can behave like the 'non-linearity' present in neural neural networks, which is a key feature of the success of neural networks. 

As such, in this challenge, you will construct a rudimentary 'quantum neural network' which can classify.

We will use two qubits as the target, and the ancilla will be measured in the Pauli-'Y' basis:
\begin{align}
Y = \left(\begin{array}{cc}
0 & -i \\
i & 0
\end{array}
\right)
\end{align}


We will apply some parameterised unitary operations, $\mathcal{U}(\theta)$ to three qubits, and then measure the ancilla in the $Y$ basis.

In this simple example, we will be classifying binary strings, $z\in \{0, 1\}^n$.

Each string will have a predefined label, $l(z)$, which is either $\pm 1$

The 'loss' function (per example string z) will be the difference between the expectation value of the ancilla when measured in the $Y$ basis, and the true label:
\begin{align}
    L(z) = 1 - l(z)\langle, z, 1|\mathcal{U}(\theta)Y_{anc}\mathcal{U}(\theta)|z, 1\rangle
\end{align}

If the value of the **true** label (say +1) is different than that of the measured one (say -1), then:
measured in the $Y$ basis, and the true label:
\begin{align}
    L(z) = l(z)\langle, z, 1|\mathcal{U}(\theta)Y_{anc}\mathcal{U}(\theta)|z, 1\rangle = (+1)(-1) = -1
\end{align}

so the cost function will be at maximum (the minimum would be where the measured value and the true label are identical so :
\begin{align}
    L(z) = l(z)\langle, z, 1|\mathcal{U}(\theta)Y_{anc}\mathcal{U}(\theta)|z, 1\rangle = 1\\
    \implies L(z) = 0
\end{align}
